In [1]:
from PyQt5 import QtWidgets, QtGui, QtCore
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LogisticRegression
import os
from sklearn.model_selection import train_test_split

class LinearRegressionGUI(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()

        # Initialize the file object to None
        self.file_obj = None

        # Create a QLabel to display the title
        self.title_label = QtWidgets.QLabel("Logistic Regression  GUI")
        self.title_label.setAlignment(QtCore.Qt.AlignCenter)
        font = QtGui.QFont()
        font.setPointSize(20)
        font.setBold(True)
        self.title_label.setFont(font)

        # Create a QGroupBox to hold the file selection widgets
        file_group_box = QtWidgets.QGroupBox("Load Data")
        file_layout = QtWidgets.QHBoxLayout()

        # Create a QLineEdit to display the file path
        self.file_path_entry = QtWidgets.QLineEdit()
        self.file_path_entry.setReadOnly(True)
        file_layout.addWidget(self.file_path_entry)

        # Create a QPushButton to select the CSV file
        select_file_button = QtWidgets.QPushButton("Select File")
        select_file_button.clicked.connect(self.select_file)
        file_layout.addWidget(select_file_button)

        file_group_box.setLayout(file_layout)

        # Create a QGroupBox to hold the parameter widgets
        parameter_group_box = QtWidgets.QGroupBox("Parameters")
        parameter_layout = QtWidgets.QVBoxLayout()

        # Create a QLabel and QSlider for the split scale parameter
        split_layout = QtWidgets.QHBoxLayout()
        split_label = QtWidgets.QLabel("Train/Test Split Scale:")
        self.split_scale = QtWidgets.QSlider(QtCore.Qt.Horizontal)
        self.split_scale.setMinimum(0)
        self.split_scale.setMaximum(10)
        self.split_scale.setSingleStep(1)
        self.split_scale.setValue(5)
        split_layout.addWidget(split_label)
        split_layout.addWidget(self.split_scale)
        parameter_layout.addLayout(split_layout)
        

            # Create QRadioButton widgets for the Models
#         self.rf = QtWidgets.QRadioButton("RF")
#         self.lr = QtWidgets.QRadioButton("LR")
#         self.rf.setChecked(True)
#         model_layout = QtWidgets.QHBoxLayout()
#         model_layout.addWidget(self.rf)
#         model_layout.addWidget(self.lr)
#         parameter_layout.addLayout(model_layout)

        # Create a QLabel and QLineEdit for the lambda parameter
        lambda_layout = QtWidgets.QHBoxLayout()
        lambda_label = QtWidgets.QLabel("Lambda:")
        self.lambda_entry = QtWidgets.QLineEdit()
        lambda_layout.addWidget(lambda_label)
        lambda_layout.addWidget(self.lambda_entry)
        parameter_layout.addLayout(lambda_layout)

        # Create QRadioButton widgets for the error metric
        self.mse_radio = QtWidgets.QRadioButton("MSE")
        self.rmse_radio = QtWidgets.QRadioButton("RMSE")
        self.mae_radio = QtWidgets.QRadioButton("MAE")
        self.acc_radio = QtWidgets.QRadioButton("ACC")
        self.mse_radio.setChecked(True)
        error_layout = QtWidgets.QHBoxLayout()
        error_layout.addWidget(self.mse_radio)
        error_layout.addWidget(self.rmse_radio)
        error_layout.addWidget(self.mae_radio)
        error_layout.addWidget(self.acc_radio)
        parameter_layout.addLayout(error_layout)

        # Add the parameter widgets to the group box
        parameter_group_box.setLayout(parameter_layout)

        # Create a QPushButton to calculate the linear regression
        calculate_button = QtWidgets.QPushButton("Calculate")
        calculate_button.clicked.connect(self.calculate_LogisticRegression)

        # Create QLabel widgets to display the training and testing error
        self.train_error_label = QtWidgets.QLabel("Training Error: ")
        self.test_error_label = QtWidgets.QLabel("Testing Error: ")

        # Create a QTableView widget to display the file contents
        self.table_view = QtWidgets.QTableView()

        #description
        self.columns = QtWidgets.QLabel("num_Col:")
        self.rows = QtWidgets.QLabel("num_Row:")
        des_layout = QtWidgets.QHBoxLayout()
        des_layout.addWidget(self.columns)
        des_layout.addWidget(self.rows)
        self.des_group_box = QtWidgets.QGroupBox("Description")
        self.des_group_box.setLayout(des_layout)
        
        # Create a QGridLayout to hold the widgets
        layout = QtWidgets.QGridLayout()
        layout.addWidget(self.title_label, 0, 0, 1, 3)
        layout.addWidget(file_group_box, 1, 0, 1, 3)
        layout.addWidget(parameter_group_box, 2, 0, 1, 3)
        layout.addWidget(calculate_button, 3, 1)
        layout.addWidget(self.train_error_label, 4, 0, QtCore.Qt.AlignRight)
        layout.addWidget(self.test_error_label, 4, 2, QtCore.Qt.AlignLeft)
        layout.addWidget(self.table_view, 5, 0, 1, 3)
        layout.addWidget(self.des_group_box, 6, 0, 1, 3)
        
        self.setLayout(layout)

        self.setWindowTitle("Linear Regression GUI")

    def select_file(self):
        file_path, _ = QtWidgets.QFileDialog.getOpenFileName(self, "Open CSV", "", "CSV Files (*.csv)")
        if file_path:
            self.load_data_from_csv(file_path)

    def load_data_from_csv(self, file_path):
        try:
            df = pd.read_csv(file_path)
            self.file_path_entry.setText(file_path)
            self.file_obj = df

            # Display the file contents in a table
            model = QtGui.QStandardItemModel(df.shape[0], df.shape[1])
            for row in range(df.shape[0]):
                for column in range(df.shape[1]):
                    item = QtGui.QStandardItem(str(df.iloc[row, column]))
                    model.setItem(row, column, item)
            self.table_view.setModel(model)
            numcol=len(df.columns)
            numrow=len(df)
            file_name =os.path.basename(file_path)
            self.columns.setText(f"num_Col: {numcol:.2f}")
            self.rows.setText(f"num_Row: {numrow:.2f}")
            self.des_group_box.setTitle(file_name)
            return df
        except FileNotFoundError:
            print(f"ERROR: Could not load file {file_path}")

    
    


    def calculate_LogisticRegression(self):
        if self.file_obj is None:
            print("ERROR: No file selected")
            return

        # Split the data into training and testing sets
        X = self.file_obj.iloc[:, :-1]
        y = self.file_obj.iloc[:, -1]
        split_scale = self.split_scale.value() / 10
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split_scale, random_state=42)

        # Perform linear regression on the training data
        alpha_text = self.lambda_entry.text()
        if not alpha_text:
            print("ERROR: Lambda value is empty")
            return
        alpha = float(alpha_text)
        lr = LogisticRegression(C=alpha)
        lr.fit(X_train, y_train)

        # Calculate the training error
        y_train_pred = lr.predict(X_train)
        mse_train = mean_squared_error(y_train, y_train_pred)
        rmse_train = mse_train ** 0.5
        mae_train = mean_absolute_error(y_train, y_train_pred)

        # Calculate the testing error
        y_test_pred = lr.predict(X_test)
        mse_test = mean_squared_error(y_test, y_test_pred)
        rmse_test = mse_test ** 0.5
        mae_test = mean_absolute_error(y_test, y_test_pred)

        # Check which error metric is selected and update the GUI accordingly
        if self.mse_radio.isChecked():
            self.train_error_label.setText(f"Training Error: MSE={mse_train:.2f}")
            self.test_error_label.setText(f"Testing Error: MSE={mse_test:.2f}")
        elif self.rmse_radio.isChecked():
            self.train_error_label.setText(f"Training Error: RMSE={rmse_train:.2f}")
            self.test_error_label.setText(f"Testing Error: RMSE={rmse_test:.2f}")
        elif self.mae_radio.isChecked():
            self.train_error_label.setText(f"Training Error: MAE={mae_train:.2f}")
            self.test_error_label.setText(f"Testing Error: MAE={mae_test:.2f}")
        else:
            print("No radio button is selected")


if __name__ == "__main__":
    app = QtWidgets.QApplication([])
    gui = LinearRegressionGUI()
    gui.show()
    app.exec_()